In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
from pyspark.sql.functions import substring, avg, sum,max,min,col

# read data from the silver tables
air_quality = spark.table("PL7G1_SILVER.air_quality")

In [27]:
# create silver database
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS PL7G1_GOLD LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/'
    """
)

DataFrame[]

In [2]:
spark.sql(
    """
    Drop DATABASE PL7G1_GOLD CASCADE
    """
)

DataFrame[]

In [4]:
pivot_Name = air_quality.filter((air_quality.Name == "Ozone (O3)") | (air_quality.Name == "Sulfur Dioxide (SO2)") | (air_quality.Name == "Fine Particulate Matter (PM2.5)") | (air_quality.Name == "Nitrogen Dioxide (NO2)"))

pivot_Name = pivot_Name.groupBy("Year","Borough").pivot("Name").agg(
sum(col("Data_Value")).alias("sum"),
avg(col("Data_Value")).alias("avg"),
min(col("Data_Value")).alias("min"),
max(col("Data_Value")).alias("max"))



pivot_Name.toPandas()

Year         Borough  Fine Particulate Matter (PM2.5)_sum  \
0   2018       Manhattan                               735.91   
1   2010  Sem Informação                               261.62   
2   2016        Citywide                                15.99   
3   2010   Staten Island                               261.57   
4   2016       Manhattan                               495.79   
..   ...             ...                                  ...   
86  2014       Manhattan                               568.61   
87  2010           Bronx                               887.72   
88  2019   Staten Island                               161.87   
89  2010        Citywide                                34.49   
90  2018        Citywide                                22.57   

    Fine Particulate Matter (PM2.5)_avg  Fine Particulate Matter (PM2.5)_min  \
0                              9.085309                                 7.32   
1                             12.458095                                 9.42   
2                              7.995000                                 7.89   
3                             10.898750                                 9.29   
4                              9.181296                                 8.17   
..                                  ...                                  ...   
86                            10.529815                                 8.88   
87                            12.329444                                10.41   
88                             6.744583                                 5.81   
89                            11.496667                                10.10   
90                             7.523333                                 7.00   

    Fine Particulate Matter (PM2.5)_max  Nitrogen Dioxide (NO2)_sum  \
0                                 13.46                     1859.42   
1                                 15.38                      561.42   
2                                  8.10                       37.32   
3                                 12.16                      430.74   
4                                 11.74                     1353.18   
..                                  ...                         ...   
86                                13.20                     1381.86   
87                                15.05                     1722.06   
88                                 7.79                      334.10   
89                                12.56                       67.24   
90                                 8.32                       49.66   

    Nitrogen Dioxide (NO2)_avg  Nitrogen Dioxide (NO2)_min  \
0                    22.955802                       14.61   
1                    26.734286                       10.82   
2                    18.660000                       13.20   
3                    17.947500                        9.42   
4                    25.058889                       15.78   
..                         ...                         ...   
86                   25.590000                       16.63   
87                   23.917500                       16.69   
88                   13.920833                        7.75   
89                   22.413333                       17.59   
90                   16.553333                       12.32   

    Nitrogen Dioxide (NO2)_max  Ozone (O3)_sum  Ozone (O3)_avg  \
0                        31.49          730.54       27.057037   
1                        38.87          215.64       30.805714   
2                        24.12           33.82       33.820000   
3                        27.72          277.89       34.736250   
4                        36.31          819.55       30.353704   
..                         ...             ...             ...   
86                       35.01          731.26       27.083704   
87                       32.21          786.49       32.770417   
88                       21.89          231.91       28.988750   
89  

In [5]:
pivot_Name.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Fine Particulate Matter (PM2.5)_sum: double (nullable = true)
 |-- Fine Particulate Matter (PM2.5)_avg: double (nullable = true)
 |-- Fine Particulate Matter (PM2.5)_min: double (nullable = true)
 |-- Fine Particulate Matter (PM2.5)_max: double (nullable = true)
 |-- Nitrogen Dioxide (NO2)_sum: double (nullable = true)
 |-- Nitrogen Dioxide (NO2)_avg: double (nullable = true)
 |-- Nitrogen Dioxide (NO2)_min: double (nullable = true)
 |-- Nitrogen Dioxide (NO2)_max: double (nullable = true)
 |-- Ozone (O3)_sum: double (nullable = true)
 |-- Ozone (O3)_avg: double (nullable = true)
 |-- Ozone (O3)_min: double (nullable = true)
 |-- Ozone (O3)_max: double (nullable = true)
 |-- Sulfur Dioxide (SO2)_sum: double (nullable = true)
 |-- Sulfur Dioxide (SO2)_avg: double (nullable = true)
 |-- Sulfur Dioxide (SO2)_min: double (nullable = true)
 |-- Sulfur Dioxide (SO2)_max: double (nullable = true)



In [6]:
Renamed = pivot_Name.toDF('Year', 'Borough', 'PM2_5_SUM', 'PM2_5_AVG', 'PM2_5_MIN', 'PM2_5_MAX', 'NO2_SUM', 'NO2_AVG', 'NO2_MIN', 'NO2_MAX', 'O3_SUM', 'O3_AVG', 'O3_MIN', 'O3_MAX', 'SO2_SUM', 'SO2_AVG', 'SO2_MIN', 'SO2_MAX')

In [7]:
Renamed.toPandas()

Year         Borough  PM2_5_SUM  PM2_5_AVG  PM2_5_MIN  PM2_5_MAX  NO2_SUM  \
0   2018       Manhattan     735.91   9.085309       7.32      13.46  1859.42   
1   2010  Sem Informação     261.62  12.458095       9.42      15.38   561.42   
2   2016        Citywide      15.99   7.995000       7.89       8.10    37.32   
3   2010   Staten Island     261.57  10.898750       9.29      12.16   430.74   
4   2016       Manhattan     495.79   9.181296       8.17      11.74  1353.18   
..   ...             ...        ...        ...        ...        ...      ...   
86  2014       Manhattan     568.61  10.529815       8.88      13.20  1381.86   
87  2010           Bronx     887.72  12.329444      10.41      15.05  1722.06   
88  2019   Staten Island     161.87   6.744583       5.81       7.79   334.10   
89  2010        Citywide      34.49  11.496667      10.10      12.56    67.24   
90  2018        Citywide      22.57   7.523333       7.00       8.32    49.66   

      NO2_AVG  NO2_MIN  NO2_MAX  O3_SUM     O3_AVG  O3_MIN  O3_MAX  SO2_SUM  \
0   22.955802    14.61    31.49  730.54  27.057037   22.91   29.02      NaN   
1   26.734286    10.82    38.87  215.64  30.805714   26.19   35.76    35.35   
2   18.660000    13.20    24.12   33.82  33.820000   33.82   33.82      NaN   
3   17.947500     9.42    27.72  277.89  34.736250   32.74   36.53    14.80   
4   25.058889    15.78    36.31  819.55  30.353704   26.92   32.01      NaN   
..        ...      ...      ...     ...        ...     ...     ...      ...   
86  25.590000    16.63    35.01  731.26  27.083704   21.79   29.67    31.25   
87  23.917500    16.69    32.21  786.49  32.770417   30.97   35.04   142.49   
88  13.920833     7.75    21.89  231.91  28.988750   27.27   30.14      NaN   
89  22.413333    17.59    28.07   33.68  33.680000   33.68   33.68     3.63   
90  16.553333    12.32    20.34   30.56  30.560000   30.56   30.56      NaN   

     SO2_AVG  SO2_MIN  SO2_MAX  
0        NaN      NaN      NaN  
1   5.050000     1.66     9.51  
2        NaN      NaN      NaN  
3   1.850000     1.52     2.13  
4        NaN      NaN      NaN  
..       ...      ...      ...  
86  1.157407     0.55     1.55  
87  5.937083     3.62    10.52  
88       NaN      NaN      NaN  
89  3.630000     3.63     3.63  
90       NaN      NaN      NaN  

[91 rows x 18 columns]

In [24]:
spark.sql(
    """
    DROP TABLE IF EXISTS PL7G1_GOLD.Air_Quality_Gases
    """
)

DataFrame[]

In [8]:
spark.sql(
    """
    CREATE EXTERNAL TABLE PL7G1_GOLD.Air_Quality_Gases (
        Year VARCHAR(5),
        Borough VARCHAR(50),
        PM2_5_SUM Double,
        PM2_5_AVG Double,
        PM2_5_MIN Double,
        PM2_5_MAX Double,
        NO2_SUM Double,
        NO2_AVG Double,
        NO2_MIN Double,
        NO2_MAX Double,
        O3_SUM Double,
        O3_AVG Double,
        O3_MIN Double,
        O3_MAX Double,
        SO2_SUM Double,
        SO2_AVG Double,
        SO2_MIN Double,
        SO2_MAX Double
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Air_Quality_Gases'
    """
)

DataFrame[]

In [9]:
Renamed.toPandas()

Year         Borough  PM2_5_SUM  PM2_5_AVG  PM2_5_MIN  PM2_5_MAX  NO2_SUM  \
0   2018       Manhattan     735.91   9.085309       7.32      13.46  1859.42   
1   2010  Sem Informação     261.62  12.458095       9.42      15.38   561.42   
2   2016        Citywide      15.99   7.995000       7.89       8.10    37.32   
3   2010   Staten Island     261.57  10.898750       9.29      12.16   430.74   
4   2016       Manhattan     495.79   9.181296       8.17      11.74  1353.18   
..   ...             ...        ...        ...        ...        ...      ...   
86  2014       Manhattan     568.61  10.529815       8.88      13.20  1381.86   
87  2010           Bronx     887.72  12.329444      10.41      15.05  1722.06   
88  2019   Staten Island     161.87   6.744583       5.81       7.79   334.10   
89  2010        Citywide      34.49  11.496667      10.10      12.56    67.24   
90  2018        Citywide      22.57   7.523333       7.00       8.32    49.66   

      NO2_AVG  NO2_MIN  NO2_MAX  O3_SUM     O3_AVG  O3_MIN  O3_MAX  SO2_SUM  \
0   22.955802    14.61    31.49  730.54  27.057037   22.91   29.02      NaN   
1   26.734286    10.82    38.87  215.64  30.805714   26.19   35.76    35.35   
2   18.660000    13.20    24.12   33.82  33.820000   33.82   33.82      NaN   
3   17.947500     9.42    27.72  277.89  34.736250   32.74   36.53    14.80   
4   25.058889    15.78    36.31  819.55  30.353704   26.92   32.01      NaN   
..        ...      ...      ...     ...        ...     ...     ...      ...   
86  25.590000    16.63    35.01  731.26  27.083704   21.79   29.67    31.25   
87  23.917500    16.69    32.21  786.49  32.770417   30.97   35.04   142.49   
88  13.920833     7.75    21.89  231.91  28.988750   27.27   30.14      NaN   
89  22.413333    17.59    28.07   33.68  33.680000   33.68   33.68     3.63   
90  16.553333    12.32    20.34   30.56  30.560000   30.56   30.56      NaN   

     SO2_AVG  SO2_MIN  SO2_MAX  
0        NaN      NaN      NaN  
1   5.050000     1.66     9.51  
2        NaN      NaN      NaN  
3   1.850000     1.52     2.13  
4        NaN      NaN      NaN  
..       ...      ...      ...  
86  1.157407     0.55     1.55  
87  5.937083     3.62    10.52  
88       NaN      NaN      NaN  
89  3.630000     3.63     3.63  
90       NaN      NaN      NaN  

[91 rows x 18 columns]

In [10]:
# write to delta table
Renamed \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Air_Quality_Gases")

In [11]:
spark.table("PL7G1_GOLD.Air_Quality_Gases").toPandas()

Year        Borough  PM2_5_SUM  PM2_5_AVG  PM2_5_MIN  PM2_5_MAX  NO2_SUM  \
0   2008  Staten Island     174.02  10.876250       9.63      12.46   320.24   
1   2012  Staten Island     221.72   9.238333       7.82      10.13   367.51   
2   2016  Staten Island     119.87   7.491875       6.87       8.22   233.28   
3   2016      Manhattan     495.79   9.181296       8.17      11.74  1353.18   
4   2015  Staten Island     248.02   7.750625       6.59       9.08   453.51   
..   ...            ...        ...        ...        ...        ...      ...   
86  2016         Queens     522.19   7.911970       6.87       9.77  1259.33   
87  2020         Queens     429.23   6.503485       5.33       7.90   835.39   
88  2019          Bronx     527.34   7.324167       6.33       9.00  1267.92   
89  2018          Bronx     563.27   7.823194       6.88       9.50  1273.45   
90  2008          Bronx     609.82  12.704583      10.47      15.65  1322.26   

      NO2_AVG  NO2_MIN  NO2_MAX   O3_SUM     O3_AVG  O3_MIN  O3_MAX  SO2_SUM  \
0   20.015000    14.70    26.45      NaN        NaN     NaN     NaN    17.70   
1   15.312917     8.03    23.64   266.48  33.310000   31.06   35.27     3.57   
2   14.580000     6.60    21.50   278.26  34.782500   33.40   36.19      NaN   
3   25.058889    15.78    36.31   819.55  30.353704   26.92   32.01      NaN   
4   14.172188     6.59    21.29   257.10  32.137500   29.99   34.09     0.96   
..        ...      ...      ...      ...        ...     ...     ...      ...   
86  19.080758     6.10    27.56  1121.54  33.986061   32.70   38.26      NaN   
87  12.657424     5.90    19.22  1007.92  30.543030   29.45   31.83      NaN   
88  17.610000    10.37    26.15   709.89  29.578750   28.43   31.45      NaN   
89  17.686806    11.68    25.02   732.90  30.537500   29.53   32.04      NaN   
90  27.547083    21.33    34.05      NaN        NaN     NaN     NaN   172.98   

    SO2_AVG  SO2_MIN  SO2_MAX  
0   2.21250     1.80     2.56  
1   0.44625     0.39     0.50  
2       NaN      NaN      NaN  
3       NaN      NaN      NaN  
4   0.12000     0.11     0.13  
..      ...      ...      ...  
86      NaN      NaN      NaN  
87      NaN      NaN      NaN  
88      NaN      NaN      NaN  
89      NaN      NaN      NaN  
90  7.20750     4.43    12.69  

[91 rows x 18 columns]

In [12]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Air_Quality_Gases/`
""").show()

++
||
++
++



In [13]:
spark.sql("""
    CREATE EXTERNAL TABLE IF NOT EXISTS PL7G1_GOLD.Air_Quality_Gases_presto (
        Year VARCHAR(5),
        Borough VARCHAR(50),
        PM2_5_SUM Double,
        PM2_5_AVG Double,
        PM2_5_MIN Double,
        PM2_5_MAX Double,
        NO2_SUM Double,
        NO2_AVG Double,
        NO2_MIN Double,
        NO2_MAX Double,
        O3_SUM Double,
        O3_AVG Double,
        O3_MIN Double,
        O3_MAX Double,
        SO2_SUM Double,
        SO2_AVG Double,
        SO2_MIN Double,
        SO2_MAX Double
    )
    ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
    STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
    OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Air_Quality_Gases/_symlink_format_manifest/'
    """).show()

++
||
++
++

